In [ ]:
%pip install duckdb
%pip install notebook
%pip install jupysql pandas matplotlib duckdb-engine

You should consider upgrading via the '/Users/jarred.luzada/.asdf/installs/python/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/jarred.luzada/.asdf/installs/python/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/jarred.luzada/.asdf/installs/python/3.10.0/bin/python3.10 -m pip install --upgrade pip' command.


In [1]:
import os
osm_dataset="cemetery"
input_path = f"./data/geojson/{osm_dataset}.geojson"
input_filename = os.path.basename(input_path)

In [2]:
import duckdb
from sqlalchemy.engine import create_engine

%load_ext sql
conn = create_engine('duckdb:///data/local.duckdb')
%sql conn --alias duckdb
%config SqlMagic.named_parameters="enabled" 

In [3]:
%%sql
INSTALL psql FROM community;
INSTALL spatial;
LOAD spatial;
LOAD psql;

Running query in 'duckdb'

Success


In [4]:
%%sql
CREATE OR REPLACE TABLE point_data AS
SELECT *
FROM ST_Read("{{input_path}}")

Running query in 'duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [5]:
import random
import string

try:
    conn.raw_connection().remove_function('random_string')
except:
    pass

# Define the Python function to generate a random string of length 'n'
def random_string(n: int) -> str:
    return ''.join(random.choices(string.ascii_letters + string.digits, k=n))

# Register the function with DuckDB

conn.raw_connection().create_function('random_string', random_string, side_effects=True)

In [6]:
%%sql
CREATE OR REPLACE TABLE point_data_centroid AS
WITH centroid AS (
    FROM point_data
    SELECT centroid: geom.ST_Centroid(),
)
FROM centroid
SELECT
    id: ROW_NUMBER() OVER (ORDER BY centroid),
    locationX: centroid.ST_X(),
    locationy: centroid.ST_Y(),
    description: random_string(250)


Running query in 'duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Count


In [7]:
%sql SELECT * FROM point_data_centroid

Running query in 'duckdb'

id,locationX,locationy,description
1,71.60207635350525,54.78643014239917,zveBXtAUd2XaeBj9UMyFLIaRCCVs66W74P178juO70f7DUTo6U6COKnJCzBwXFWwAxoH73inmKb8ti3xZ50Eff7avO2KVUtfQWrUnpvWNtGCNVCQWvAAxpgxEMoH3bEHzFSycM2FlEbm3jLxk3LiDbm0Qtt6ZXcdib5k9jfM10eIMJ0HTsLMOGX5xcAH3WbADH85tkw0okUcbxftb89rgTI6wlnwOopr98NRKS9hVaMHic132zf6R3z3ev
2,68.07547055464238,40.23959944486414,3OldD765yh0M25F5d6ZNJemmphHpBrBARswIr4wNBF57mB2RVDde09Q6lZQ5PxIjOv5jLQqxOTtTG1e4ENx1reSmOwyuaphRD7X3KaVQCRbB9qkbVMMgC0uNrAj3eAxpsHzU6k0hQh7mSHn77Cqj2TC8Xk1bs7UAdk9gu3uJdOUY8DEBHyBmOmuwn4QL1msPbtBdwVkH7SytdLpCbJledsHXzDcmCPyiMwCFK0DmciW8I9HKDJjlc4kwOy
3,22.71972581720911,48.451325419849496,cY5AJU4vVEzfQRsOa9V6goRdhG1Hp480dyL0wPhbEAB0zag9i9mZIT7BAjVkglDS1iXpuJ6p5nhj9IG9ZwEHqDj1QMecmXb3rmz6dKj5T0HVVwNlbqSTRHzlGp69HeoZ52HKMxoez8vI7vlytULMVvKpukCHSxKn4fdqtVJ70fU9lDOhipXJ9Ap3WJ4JSI5bS4eWYAi0nq3zum3Q5XNEaki8xO3cpLcxnRSMf7sfAshU7atqxvhfr0suSZ
4,29.491294654319063,46.07657549119183,JQ8EgdG0dIZuPfZWQM5h6z1VlxpV6QFj1QMEyru8uv1sVLX2cZlfWNlLJXELxwFxB5tiRGxrOOeoOoAa4YaP7t62vOCGsWj277g1EYlVtwcTshWxREQPJ5GtpDiWnPOCjui5gvLCxFHKEVruOYYEn4V2Ai8pmt1DrNxvQEXfPH7WBuYkJSTabvmmj34JLdHqlEIEYwhM2rxZ63yK5w6kmspArCKmOOXQIBGZ4MMUuOTt6enr8Hfmp7wgWA
5,79.38921734504402,52.74566659600047,AjvM268rQWw6DooOn7yFosSaLNufzbKtoyewcDmNtMI7bQxZPJQbxUwasIe0C6gjfPlT79TBwJlEi0PF6v6wgGMGnmoSrZFHIQZ3s8hqNmmrfE28l7iDVaUsHSYLoA5q6pIf0yoW1NTO1439T9GqZ6BbituFREVDKS43GeMnOAPvuWbHnkdvqj1U9IHS0zeTsOy3LWb3uMEY3xqCOYBUTTxcYSGrlK4vuWEodDXf90axqarfNIt2wm4voH
6,6.359217615158059,43.142364551113644,zm71HyuBHLWweVPzIFUFASheiG9M5Vp173UVprecxHlZhtMXa2gUYyieHxnrNApVY1pvyiHSWZFAY21d18qgkwnfyH3jCuzoHGLWMQtDx9GcM0PcqpG11RgbQJ584mL8M6G9wYxl8Fs7uuJIG5i6PeDeUaxI4jYNehKpPibiuoVTb4JyvslndF6k3p4y8RMQUS33Rdc8o1omA0Vq0sOlKeAfvRDGclDu2qlqB5RmczsmCc819YcT3MzTfv
7,1.9459106666618595,49.15726517327247,TzOCLhjWWYk2edxAcxxomSMLn4M1LqqVuVQOcYzNIR4XBzsq0GcUJGlVNcErIcHainufhGjM0Y523EhanQx6mTr4FkxL1VdHwnc8yXb5ofz0k3UrnuKobsY0awryTW7gKYkLFG26hGiNAtPTPznaG9Vlz2abQhPUS0Jm9DXVxvaDdcEfRZFvXpGxAxhdgyCedprwsbVk4YIgpW79pxm2PxWAWbTN5nIZdiFCyUnTD5ywCIrt9oZwt3D5Zo
8,7.758160430065118,43.808794503076086,RG6BG7S68kEgMlNao9quqP5Q0QLlbjQNjm2wBk92pENP3HvqOU4mvskWLpXarxi429Fn4HkAxnyspyflgpeZuHWK3P2B9cNetsSkigvqlFTEMO27VMVGk8ioZW0rkLiEXrQuADVBxMivUeYJDgg8lyWs1eJPSNiZ8tTgjxXjb3NXxwswgnmJZIYsZnEpAWBJ25lJTrE1OuQVrFs2twsMnKh2A5maHYnIWvIaQTg5131UO9YGwTjKIEtatw
9,-0.9758063970803335,51.474531240130844,qviYsz5jaaian47KqH8iMIV8FPaNrnm0nAa3696eRE00kzy49ciK4ynND2KfSld07Mntl6bBjLqYbcyE9zrmo423LGTV7ftvggtdd1nqu0COwzPJevfbNYjcvJlDONlTX8Lp2rIknlWcM1vbaVfl0PhFC8nRcMBWPBfiZjSrZwQeXNMr50zqhsmw12KkY079cpzZqSTg4AQMXxXhdVFqNn01NXsDeqKm2O4LzKJAkF1sumIUoDiaZrEfLu
10,66.95163331926233,39.698323306344484,uWDh46pLH3IDMibdV7tDPSgNwnwmRu8E7W9GC47zHWyFA5eUbgfcADQ2zUZP8sZuxqEUFkOgF94I07EDXOaPOvnd2ue0FY0VsRVrpfmLQTymxOydj555CmVltCPJtfGQDsEvp0mgh1ExLQ2oTZyYwC7ci18MwPyNZTfzp3ggPQxmi0wyANoKBonha4rXsDrkQcBGOnX2TdqDTwiVnQYqQo3XfenOMRq58MltyV3nXdlOoCyzHsAGnpfKgB


In [8]:
%%sql
SELECT COUNT(*) FROM point_data_centroid as data;

Running query in 'duckdb'

count_star()
402325


In [9]:
%%sql
SUMMARIZE point_data_centroid

Running query in 'duckdb'

column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
id,BIGINT,1,402325,377825,201163.0,116141.3678633357,100582,201416,301744,402325,0.00
locationX,DOUBLE,-179.12087778314643,179.80886915,326174,1.0731648788767731,60.20403126591039,-45.87195652744291,9.42516138974874,28.598006608720496,402325,0.00
locationy,DOUBLE,-66.52681067468725,78.92123011264881,477667,38.35798860494098,20.333799419387596,36.8145559365445,44.58551598968783,49.95163493199449,402325,0.00
description,VARCHAR,0013P35QYrb92lyIsdFTnCiB5Sw61DhbxL1kuY5bdZzRwbw9NnDsSkHzJh7I269OvzkUH2Ff46jQYFc9SqnZu8WS8284CkhjQjmgRCDAC0rchpNrmqTOHvqP6ljBVfyZJBasdPFyQ4jtMFs0qCQ8r5UR3AmMMD7SGXr2Ylviodluvhyudl4yzHnxWrhL8rkPzIyaT3Gf3rCR7KsZSIUTiXOwyVMiJDNc30LbSXiF9xTilMQX69tlawyaV1,zzzHd8yUgsUogzit7oqbw2MVOdPs9rvQlsbPBKZ07qOrt59QWPGVaMl7n7zhZIkG4KAinQUEoC3HXcfuqKUQBaKX0RqmmXIxGeGjm7PH9b7QrW2vPGHKvDM40qofHvaLIYD9wrEQ4l3RxIdqgHwvkdsbRVJvcCAdCLvE73lPIW8zIwvPq8sTHGUI0UKE2nQQuNeqxMEueWSpQwc4B32gFUQBtBzIH5uDp38UWFSRpIhxyxKxLDtyWmxSCR,391516,None,None,None,None,None,402325,0.00


In [10]:
export_filename = input_filename.replace(".geojson", ".csv")
export_path = "./data/csv/" + export_filename

os.makedirs(os.path.dirname(export_path), exist_ok=True)

print(f"Exporting to {export_filename}")
print(f"Exporting to {export_path}")

Exporting to cemetery.csv
Exporting to ./data/csv/cemetery.csv


In [11]:
%%sql
COPY (SELECT * FROM point_data_centroid) TO '{{export_path}}' (FORMAT CSV, HEADER);

Running query in 'duckdb'

Count
